© HeadFirst AI, 2020

In [ ]:
import gensim
import warnings

warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning,module='gensim')  

model = gensim.models.KeyedVectors.load_word2vec_format('./data/vectors.bin.gz', binary=True)

dog = model['dog']
print(dog.shape)
print(dog[:10])

# Deal with an out of dictionary word: Михаил (Michail)
if 'Михаил' in model:
    print(model['Михаил'].shape)
else:
    print('{0} is an out of dictionary word'.format('Михаил'))


# Some predefined functions that show content related information for given words
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

print(model.doesnt_match("breakfast cereal dinner lunch".split()))

print(model.similarity('woman', 'man'))